In [2]:
# import PyPDF2
import pandas as pd
import os
import requests  ## for getting data from a server GET
import re   ## for regular expressions
import numpy as np

In [3]:
# assign directory
directory = 'events-by-state'
file_list = []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        file_list.append(f)

In [4]:
# reader = PyPDF2.PdfReader('us-climate-disasters.pdf')
cols = ["Name","Disaster","Begin Date","End Date","Total CPI-Adjusted Cost (Millions of Dollars)","Deaths","State"]
df = pd.DataFrame(columns = cols)

for file in file_list:
    df_new = pd.read_csv(file,skiprows=1)
    state = file.split('-')[3]
    df_new['State'] = [state]*len(df_new.index)
    for i_row in range(len(df_new.index)):
        search = df[df['Name']==df_new.loc[i_row,'Name']]
        if not search.empty:
            df.loc[search.index[0],'State']+= f",{state}"
        else:
            df.loc[len(df.index)] = df_new.loc[i_row,:]

# df = df.drop_duplicates(subset='Name',keep=False)
df

,Name,Disaster,Begin Date,End Date,Total CPI-Adjusted Cost (Millions of Dollars),Deaths,State
0,Western Fire Season (Fall 2002),Wildfire,20020901,20021130,2235.6,21,"AK,AL,AZ,CA,CO,GA,NM,NV,OR,UT"
1,California Wildfires (Fall 2003),Wildfire,20030901,20031130,6419.2,22,"AK,AZ,CA,ID,MN,MT,OK,OR,UT,WA"
2,Numerous Wildfires (2006),Wildfire,20060101,20061231,2259.1,28,"AK,AZ,CA,FL,ID,MS,MT,NM,NV,OK,OR,SD,TX,UT,WA,WY"
3,Western Wildfires (Summer 2007),Wildfire,20070601,20070831,4029.7,12,"AK,AZ,CA,FL,GA,ID,MN,MT,NV,OR,UT,WA"
4,Western Wildfires (Summer-Fall 2009),Wildfire,20090601,20091130,1436.8,10,"AK,AZ,CA,CO,NM,OK,TX"
...,...,...,...,...,...,...,...
363,South Texas Hail Storms (May 2020),Severe Storm,20200527,20200527,1666.0,0,TX
364,Hurricane Hanna (July 2020),Tropical Cyclone,20200725,20200726,1257.8,0,TX
365,Texas Hail Storms (April 2021),Severe Storm,20210412,20210415,1696.3,0,TX
366,Texas Hail Storms (February 2022),Severe Storm,20220221,20220222,1088.6,0,TX


In [99]:
df.to_csv(f'events-with-states', sep=',', index=False, encoding='utf-8')

In [5]:
states_convert = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}



In [6]:
query_list = {"Name": [], "Year": [], "State": []}
# df['Begin Date'] = pd.to_dateonly(df['Begin Date'])
# df['End Date'] = pd.to_datetime(df['End Date'])

for i_row in range(len(df.index)):
    states = df.loc[i_row,"State"].split(',')
    year = df.loc[i_row,'End Date']//10000
    if len(states) <= 3 and (year > 2009 and year < 2019) or year == 2021:
        for state in states:
            query_list["Name"].append(df.loc[i_row,'Name']+' before')
            query_list["Year"].append(year)
            query_list["State"].append(states_convert[state])

            query_list["Name"].append(df.loc[i_row,'Name']+ ' after')
            query_list["Year"].append(year+1)
            query_list["State"].append(states_convert[state])


In [181]:
# # reader = PyPDF2.PdfReader('us-climate-disasters.pdf')
# # cols = ["Name","Disaster","Begin Date","End Date","Total CPI-Adjusted Cost (Millions of Dollars)","Deaths","State"]
# df = pd.DataFrame(columns = cols)

# for file in file_list:
#     df_new = pd.read_csv(file,skiprows=1)
#     state = file.split('-')[3]
#     df_new['State'] = [state]*len(df_new.index)
#     df = pd.concat([df, df_new], axis=0)

# # df = df.drop_duplicates(subset='Name',keep=False)
# df
# df.to_csv(f'events-by-state-with-dup.csv', sep=',', index=False, encoding='utf-8')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [7]:
df_state_fips = pd.read_json('state-fips.json')
df_state_fips.columns=df_state_fips.iloc[0]
df_state_fips.drop([0], inplace = True)

In [8]:
cols_2017 = ['Disaster Name','Year','State','S1903_C03_001E','S1903_C03_002E','S1903_C03_003E','S1903_C03_004E','S1903_C03_005E',\
           'S1903_C03_006E','S1903_C03_007E','S1903_C03_008E''S1903_C03_009E','S1903_C03_010E',\
           'S1903_C03_011E','S1903_C03_012E','S1903_C03_013E','S1903_C03_014E']

cols_2016 = ['Disaster Name','Year','State','S1903_C02_001E','S1903_C02_002E','S1903_C02_003E','S1903_C02_004E','S1903_C02_005E',\
           'S1903_C02_006E','S1903_C02_007E','S1903_C02_008E''S1903_C02_009E','S1903_C02_010E',\
           'S1903_C02_012E','S1903_C02_013E','S1903_C02_014E','S1903_C02_015E']

In [9]:
####################################
##
##  Step 1: Connect to the server
##          Send a query
##          Collect and clean the 
##          results
####################################

####################################################
##In the following loop, we will query thenewsapi servers
##for all the topic names in the list
## We will then build a large csv file 
## where each article is a row
##
## From there, we will convert this data
## into a labeled dataframe
## so we can train and then test our DT
## model
####################################################

####################################################
## Build the URL and GET the results
## NOTE: At the bottom of this code
## commented out, you will find a second
## method for doing the following. This is FYI.
####################################################

## This is the endpoint - the server and 
## location on the server where your data 
## will be retrieved from

## TEST FIRST!
## We are about to build this URL:
## https://newsapi.org/v2/everything?apiKey=8f4134f7d0de43b8b49f91e22100f22b&q=bitcoin


endpoint_1="https://api.census.gov/data/"
endpoint_2="/acs/acs1/subject"
api_key='448115dcad7090fc4be722bf0f1406d1c232d4fd'
get_value = "NAME,group(S1903)"

df_census_2016 = pd.DataFrame(columns = cols_2016)
df_census_2017 = pd.DataFrame(columns = cols_2017)
# print(df_census.columns)

for i in range(len(query_list['Name'])):
# for i in range(10):
        
    year = query_list['Year'][i]
    print(i)
    row = df_state_fips[df_state_fips['NAME']==query_list['State'][i]]
    state_code = row.loc[row.index[0],'state']
   
    ## Dictionary Structure
    URLPost = {'get':get_value,
                'key':api_key,
                'for':f'state:{state_code}'
    }


    payload_str = "&".join("%s=%s" % (k,v) for k,v in URLPost.items())
    endpoint = endpoint_1+str(year)+endpoint_2
    # 'format=json&key=site:dummy+type:example+group:wheel'
    response = requests.get(endpoint, params=payload_str)

    # response=requests.get(endpoint, URLPost)
    # print(response.request.url)
    print(response)
    jsontxt = response.json()
    print(jsontxt)
    if response.status_code == 200:

    ##############################################`######
        df_census = df_census_2017
        if year < 2017:
            df_census = df_census_2016
        ## Open the file for append
        # MyFILE=open(filename, "a") ## "a" for append to add stuff
        df_census.loc[len(df_census.index)] = 0   
        df_census.loc[len(df_census.index)-1,'Disaster Name'] = query_list['Name'][i]     
        df_census.loc[len(df_census.index)-1,'Year'] = year
        df_census.loc[len(df_census.index)-1,'State'] = query_list['State'][i]
        for j,col in enumerate(jsontxt[0]):
            if col in df_census.columns:
                df_census.loc[len(df_census.index)-1,col] = jsontxt[1][j]




0
<Response [200]>
[['NAME', 'GEO_ID', 'NAME', 'S1903_C01_001E', 'S1903_C01_001EA', 'S1903_C01_001M', 'S1903_C01_001MA', 'S1903_C01_002E', 'S1903_C01_002EA', 'S1903_C01_002M', 'S1903_C01_002MA', 'S1903_C01_003E', 'S1903_C01_003EA', 'S1903_C01_003M', 'S1903_C01_003MA', 'S1903_C01_004E', 'S1903_C01_004EA', 'S1903_C01_004M', 'S1903_C01_004MA', 'S1903_C01_005E', 'S1903_C01_005EA', 'S1903_C01_005M', 'S1903_C01_005MA', 'S1903_C01_006E', 'S1903_C01_006EA', 'S1903_C01_006M', 'S1903_C01_006MA', 'S1903_C01_007E', 'S1903_C01_007EA', 'S1903_C01_007M', 'S1903_C01_007MA', 'S1903_C01_008E', 'S1903_C01_008EA', 'S1903_C01_008M', 'S1903_C01_008MA', 'S1903_C01_009E', 'S1903_C01_009EA', 'S1903_C01_009M', 'S1903_C01_009MA', 'S1903_C01_010E', 'S1903_C01_010EA', 'S1903_C01_010M', 'S1903_C01_010MA', 'S1903_C01_011E', 'S1903_C01_011EA', 'S1903_C01_011M', 'S1903_C01_011MA', 'S1903_C01_012E', 'S1903_C01_012EA', 'S1903_C01_012M', 'S1903_C01_012MA', 'S1903_C01_013E', 'S1903_C01_013EA', 'S1903_C01_013M', 'S1903_C01

In [230]:
variable_key_2017 = {'S1903_C03_001E':"Income, All",
                'S1903_C03_002E':"Income, White",
                'S1903_C03_003E':"Income, Black",
                'S1903_C03_004E':"Income, Native",
                'S1903_C03_005E':"Income, Asian",
                'S1903_C03_006E':"Income, Pacific Islander",
                'S1903_C03_007E':"Income, Other",
                'S1903_C03_008E':"Income, Two",
                'S1903_C03_009E':"Income, Hispanic",
                'S1903_C03_010E':"Income, White Alone",
                'S1903_C03_011E':"Income, 15 to 24",
                'S1903_C03_012E':"Income, 25 to 44",
                'S1903_C03_013E':"Income, 45 to 64",
                'S1903_C03_014E':"Income, 65 over"}
variable_key_2016 = {'S1903_C02_001E':"Income, All",
                'S1903_C02_002E':"Income, White",
                'S1903_C02_003E':"Income, Black",
                'S1903_C02_004E':"Income, Native",
                'S1903_C02_005E':"Income, Asian",
                'S1903_C02_006E':"Income, Pacific Islander",
                'S1903_C02_007E':"Income, Other",
                'S1903_C02_008E':"Income, Two",
                'S1903_C02_009E':"Income, Hispanic",
                'S1903_C02_010E':"Income, White Alone",
                'S1903_C02_012E':"Income, 15 to 24",
                'S1903_C02_013E':"Income, 25 to 44",
                'S1903_C02_014E':"Income, 45 to 64",
                'S1903_C02_015E':"Income, 65 over"}

df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
df_census

,Disaster Name,Year,S1903_C01_001E,S1903_C01_002E,S1903_C01_003E,S1903_C01_004E,S1903_C01_005E,S1903_C01_006E,S1903_C01_007E,S1903_C01_008ES1903_C01_009E,S1903_C01_010E,S1903_C01_011E,S1903_C01_012E,S1903_C01_013E,S1903_C01_014E,S1903_C01_015E
0,Arizona Severe Weather (October 2010) before,Arizona,2334050,84.1,4.0,3.3,2.3,-999999999.0,4.6,0,68.4,0,5.0,34.3,37.8,22.9
1,Arizona Severe Weather (October 2010) after,Arizona,2356055,84.0,3.8,3.1,2.6,-999999999.0,4.7,0,67.9,4.6,33.7,37.9,23.8,1543923
2,California Flooding (February 2017) before,California,13005097,8372857,811762,103336,1808321,41521,1435380,0,6193847,354364,4413107,5136242,3101384,8910617
3,California Flooding (February 2017) after,California,13072122,8447796,814526,98660,1851359,38910,1361355,0,6157056,368565,4444303,5062037,3197217,8934621
4,Rockies/Southwest Severe Weather (June 2012) b...,Colorado,1996088,87.7,3.6,0.8,2.3,-999999999.0,3.3,0,76.8,5.1,36.1,39.7,19.1,1278264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,North Texas Hail Storm (March 2016) after,Oklahoma,1470364,1130728,106390,95345,24306,-999999999,29247,0,1064834,75515,489463,531251,374135,964144
92,North Texas Hail Storm (March 2016) before,Texas,9535612,75.9,12.7,0.5,4.3,0.1,4.8,0,50.3,4.5,37.2,37.5,20.8,6602215
93,North Texas Hail Storm (March 2016) after,Texas,9623874,7266357,1228405,46566,428516,6887,472532,0,4779209,401872,3583849,3608523,2029630,6696061
94,Hurricane Maria (September 2017) before,Puerto Rico,1191305,782616,148590,-999999999,-999999999,-999999999,189131,0,8051,26313,293799,462547,408646,805573


In [225]:
# Pivot by state occurrence for creating visualization
df_events_state = pd.read_csv("events-by-state-with-dup.csv")
df_events_state

df_state_num = pd.pivot_table(df_events_state, values='Name', index='State', aggfunc='count')
df_state_num = df_state_num.reset_index()
df_state_num.to_csv(f'state_num.csv', sep=',', index=False, encoding='utf-8')